## Test notebook for Data Management Acceptance Test Campaign 1

This test will be executed on the Rubin Science Platform Notebook Aspect at the Interim Data Facility, initialized with Science Pipelines release `w_2022_32`.

### Test case LVV-T74: Verify implementation of Template Coadds

Verify that the DMS can produce Template Coadds for DIA processing.


In [1]:
from lsst.daf.butler import Butler
import lsst.ip.diffim as ip_diffim
import lsst.afw.display as afwDisplay

# LSST package for TAP queries
from lsst.rsp import get_tap_service

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

#### Instantiate the butler and the default DP0.2 collection:

In [2]:
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collection)

#### Select a visit/detector at random for exploration, and create a dataId for retrieving data from the Butler:

In [3]:
dataId = {'instrument': 'LSSTCam-imSim', 'detector': 78, 'visit': 60891}

In [4]:
# Retrieve the calexp (PVI) and the diaSrc catalog corresponding to this dataId:

calexp = butler.get('calexp', dataId=dataId)
dia_src = butler.get('goodSeeingDiff_diaSrc', dataId=dataId)

In [5]:
print(calexp.getFilter())
print('\n', calexp.visitInfo)
print('\n BBox: ',calexp.getBBox(), '; (X0, Y0) = (',calexp.x0, ',' , calexp.y0,')')
print('\n Mean, median, and stdev of image: ', np.nanmean(calexp.image.array),
      np.nanmedian(calexp.image.array), np.nanstd(calexp.image.array))

FilterLabel(band="i", physical="i_sim_1.4")

 VisitInfo(exposureId=60891078, exposureTime=30, darkTime=30, date=2022-03-21T01:23:39.513799850, UT1=nan, ERA=3.4737 rad, boresightRaDec=(73.1449917207, -38.6867405808), boresightAzAlt=(243.7456541424, +44.1646444254), boresightAirmass=1.39867, boresightRotAngle=4.16327 rad, rotType=1, observatory=-30.2446N, -70.7494E  2663, weather=Weather(nan, nan, 40), instrumentLabel='LSSTCam-imSim', id=60891, focusZ=nan)

 BBox:  (minimum=(0, 0), maximum=(4071, 3999)) ; (X0, Y0) = ( 0 , 0 )

 Mean, median, and stdev of image:  10.30323 1.298006 442.78458


The calexp appears to be a well-formed image, and has average counts near zero (as expected for a sky-subtracted image) and std deviation ~ 400 counts.

Inspect the associated `diaSrc` catalog:

In [6]:
dia_src.asAstropy()

id,coord_ra,coord_dec,parent,flags_negative,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_PeakCentroid_x,base_PeakCentroid_y,base_PeakCentroid_flag,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_SdssCentroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_SdssCentroid_flag_resetToPeak,base_SdssCentroid_flag_badError,ip_diffim_NaiveDipoleCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianFlux_flag_badCentroid,base_NaiveCentroid_flag_badInitialCentroid,base_PeakLikelihoodFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,base_SdssCentroid_flag_badInitialCentroid,base_SdssShape_flag_badCentroid,slot_Centroid_flag,ip_diffim_NaiveDipoleCentroid_pos_flag,slot_Centroid_pos_flag,ip_diffim_NaiveDipoleCentroid_neg_flag,slot_Centroid_neg_flag,ip_diffim_NaiveDipoleCentroid_x,slot_Centroid_x,ip_diffim_NaiveDipoleCentroid_y,slot_Centroid_y,ip_diffim_NaiveDipoleCentroid_xErr,slot_Centroid_xErr,ip_diffim_NaiveDipoleCentroid_yErr,slot_Centroid_yErr,ip_diffim_NaiveDipoleCentroid_pos_x,slot_Centroid_pos_x,ip_diffim_NaiveDipoleCentroid_pos_y,slot_Centroid_pos_y,ip_diffim_NaiveDipoleCentroid_pos_xErr,slot_Centroid_pos_xErr,ip_diffim_NaiveDipoleCentroid_pos_yErr,slot_Centroid_pos_yErr,ip_diffim_NaiveDipoleCentroid_neg_x,slot_Centroid_neg_x,ip_diffim_NaiveDipoleCentroid_neg_y,slot_Centroid_neg_y,ip_diffim_NaiveDipoleCentroid_neg_xErr,slot_Centroid_neg_xErr,ip_diffim_NaiveDipoleCentroid_neg_yErr,slot_Centroid_neg_yErr,base_SdssShape_xx,slot_Shape_xx,base_SdssShape_yy,slot_Shape_yy,base_SdssShape_xy,slot_Shape_xy,base_SdssShape_xxErr,slot_Shape_xxErr,base_SdssShape_yyErr,slot_Shape_yyErr,base_SdssShape_xyErr,slot_Shape_xyErr,base_SdssShape_x,slot_Shape_x,base_SdssShape_y,slot_Shape_y,base_SdssShape_instFlux,slot_Shape_instFlux,base_SdssShape_instFluxErr,slot_Shape_instFluxErr,base_SdssShape_psf_xx,slot_PsfShape_xx,slot_Shape_psf_xx,base_SdssShape_psf_yy,slot_PsfShape_yy,slot_Shape_psf_yy,base_SdssShape_psf_xy,slot_PsfShape_xy,slot_Shape_psf_xy,base_SdssShape_instFlux_xx_Cov,slot_Shape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,slot_Shape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,slot_Shape_instFlux_xy_Cov,base_SdssShape_flag,base_GaussianFlux_flag_badShape,slot_Shape_flag,base_SdssShape_flag_unweightedBad,base_GaussianFlux_flag_badShape_unweightedBad,slot_Shape_flag_unweightedBad,base_SdssShape_flag_unweighted,base_GaussianFlux_flag_badShape_unweighted,slot_Shape_flag_unweighted,base_SdssShape_flag_shift,base_GaussianFlux_flag_badShape_shift,slot_Shape_flag_shift,base_SdssShape_flag_maxIter,base_GaussianFlux_flag_badShape_maxIter,slot_Shape_flag_maxIter,base_SdssShape_flag_psf,base_GaussianFlux_flag_badShape_psf,slot_Shape_flag_psf,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_3_0_flag,base_CircularApertureFlux_3_0_flag_apertureTruncated,base_CircularApertureFlux_3_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_4_5_flag,base_CircularApertureFlux_4_5_flag_apertureTruncated,base_CircularApertureFlux_4_5_flag_sincCoeffsTruncated,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_6_0_flag,base_CircularApertureFlux_6_0_flag_apertureTruncated,base_CircularApertureFlux_6_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_9_0_flag,base_CircularApertureFlux_9_0_flag_apertureTruncated,base_CircularApertureFlux_9_0_flag_sincCoeffsTruncated,base_CircularApertureFlux_12_0_instFlux,slot_ApFlux_instFlux,base_CircularApertureFlux_12_0_instFluxErr,slot_ApFlux_instFluxErr,base_Circ

In [7]:
photocalib = calexp.getPhotoCalib()
psfmag = photocalib.instFluxToMagnitude(dia_src, 'slot_PsfFlux')
# print(psfmag)

To confirm that the sources from the `diaSrc` catalog have been mapped to objects in the `diaObjectTable`, we will instead use a TAP query so we can join on the `objectId` column from each table.

The query selects sources from `DiaSource` based on the `ccdVisitId`, then joins on `objectId` with the `DiaObject` table.

In [8]:
service = get_tap_service()

In [ ]:
query = "SELECT obj.diaObjectId, obj.ra, obj.decl, obj.nDiaSources, "\
        "obj.iPSFluxMin, obj.iPSFluxMax, obj.iPSFluxMean, obj.iPSFluxSigma, "\
        "obj.iPSFluxMAD, obj.iPSFluxChi2, obj.iPSFluxNdata, obj.iPSFluxSkew, "\
        "obj.iTOTFluxMean, obj.iTOTFluxSigma, "\
        "scisql_nanojanskyToAbMag(obj.iTOTFluxMean) as imag_obj, "\
        "src.diaSourceId, src.diaObjectId, src.midPointTai, "\
        "src.ccdVisitId, src.filterName, src.psFlux, src.psFluxErr, "\
        "src.totFlux, src.totFluxErr, "\
        "scisql_nanojanskyToAbMag(src.totFlux) as imag, "\
        "scisql_nanojanskyToAbMag(src.psFlux) as imag_psf "\
        "FROM dp02_dc2_catalogs.DiaObject as obj "\
        "JOIN dp02_dc2_catalogs.DiaSource as src "\
        "ON src.diaObjectId = obj.diaObjectId "\
        "WHERE src.ccdVisitId = 60891078 "

results = service.search(query)
diaSources = results.to_table()

In [ ]:
diaSources

Note that the resulting catalog has the same length as the one retrieved from the butler.

In [ ]:
# Filter out NaN or empty values:

okmag_src = (np.isfinite(psfmag[:, 0]))
okmag_diaSource = (np.isfinite(diaSources['imag_psf']))

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 24,
          'font.size': 20,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [7, 7],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
plt.hist(psfmag[okmag_src, 0], bins=np.arange(15, 25, 0.5), histtype='step',
         label='butler diaSrc', linewidth=4, color='red')
plt.hist(diaSources[okmag_diaSource]['imag_psf'], bins=np.arange(15, 25, 0.5),
         histtype='step', label='TAP DiaSource', linewidth=4, color='black', linestyle='--')
plt.legend(loc='upper left')
plt.xlabel('imag (psf)')
plt.ylabel('N')
plt.show()

We see that the magnitude distributions match. (Note that the additional sources present in `DiaObject` but not in the single-visit `DiaSource` catalog are expected because `DiaObject` contains the superset of all objects detected in all difference images, while `DiaSource` refers only to the single visit.

#### Compare the template image and the calexp to confirm that they represent the same field:

In [ ]:
templ = butler.get('goodSeeingDiff_templateExp', dataId=dataId)

In [ ]:
templ

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 7))
plt.sca(ax[0])  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(templ.image)
plt.title('coadd image (template)')
plt.xlim(0, 1000)
plt.ylim(0, 1000)
plt.sca(ax[1])  # set the second axis as current
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp.image)
plt.title('calexp')
plt.xlim(0, 1000)
plt.ylim(0, 1000)
plt.tight_layout()
plt.show()

This confirms that the template coadd (left panel) has been created, and is a well-formed image.